In [1]:
import numpy as np
import pandas as pd
from docplex.mp.model import Model
import glob
from collections import Counter
import time

from vrp_eval import *
from mat_functs import *

In [2]:
def solve(data_range = 'all', n_test = 7, weighing = 'unif', dayofweek = 'DIMANCHE'):

    # initialize df of results 
    results_df = pd.DataFrame()
    results_df = pd.DataFrame(columns = ['Date','Target','Solution', 'Status', 'SD','%SD','AD','%AD', 'Gap', 'Time'])

#     days = ['DIMANCHE', 'LUNDI', 'MARDI', 'MERCREDI', 'JEUDI', 'VENDREDI', 'SAMEDI']
    days = [dayofweek]

    for day in days:

            # create list of files (same day) for test and target data extraction
            if data_range == 'all':
                file_list = [file for file in sorted(glob.glob("*.csv")) if day in file][:-5]
            elif data_range == '18':
                file_list = [file for file in sorted(glob.glob("*.csv")) if file.startswith('18') and day in file]
            elif data_range == '19':
                file_list = [file for file in sorted(glob.glob("*.csv")) if file.startswith('19') and day in file]	

            for x in range(len(file_list)-n_test, len(file_list)):
                if weighing == 'fxd':
                    train = file_list[:-n_test]
                else:
                    train = file_list[:x]
                test = file_list[x]

#                 2MM
                proba_matrix2, s_list2, idx_list2 = create3d_mat(train, test, weighing)
#                 1MM
                proba_matrix1, s_list1, idx_list1 = create_pmatrix(train, test, weighing)

                res_plan1, stat1, actual_for_comp1, gap1, sol_time1 = solve_vrp(train, test, s_list1, proba_matrix1, idx_list1, 'first', day)
                print('sd: ', eval_sd(actual_for_comp1,res_plan1)[1], 'ad: ', eval_ad(actual_for_comp1,res_plan1)[1])
                
                res_plan, stat, actual_for_comp, gap, sol_time = solve_vrp(train, test, s_list2, proba_matrix2, idx_list2, 'first', day)
#                 res_plan, stat, actual_for_comp, gap, sol_time = solve_vrp(train, test, s_list2, proba_matrix2, idx_list2, 'second', day)
                results_df.loc[len(results_df)] = [test.split(" ")[0], actual_for_comp, res_plan, stat,
                                               eval_sd(actual_for_comp,res_plan)[1], eval_sd(actual_for_comp,res_plan)[2],
                                               eval_ad(actual_for_comp,res_plan)[1], eval_ad(actual_for_comp,res_plan)[2], gap, sol_time]
            
    return results_df, actual_for_comp, res_plan1, res_plan, proba_matrix1, proba_matrix2


In [3]:
# dayofweek = 'DIMANCHE'
# weighing = 'exp'

# results_df, actual, res1, res2, mat1, mat2 = solve('all', 1, weighing, dayofweek)

In [4]:
def solve_vrp(train, test, stop_list, proba_matrix, idx_list, order, day):
        
        print(test)
        all_stops_list = stop_list

        actual = convert(get_route(test),all_stops_list)

        n = len(set(flatten(actual)))-1 # len(stop_list)-1
        
        N = [i for i in range(1, n+1)]
        rt_count = len(actual)

#        Q = n
        
        if day == 'MARDI':
            Q = 14
        elif day == 'SAMEDI':
            Q = 25 # 20?
        else:
            Q = 13

        # add demands
        cap = []
        
        for i in range(len(stop_list)):
            if any(['DATS' in stop_list[i], 'Dats' in stop_list[i]]):
                cap.append(1)
            elif any(['TFCO LH' in stop_list[i], 'TFCO WOO' in stop_list[i], 'TFCO IXL' in stop_list[i]]):
                cap.append(4)
            elif 'ST CATH' in stop_list[i]:
                if day == 'DIMANCHE' or day == 'LUNDI':
                    cap.append(12)
                else:
                    cap.append(10)
            elif any(['TONGRES' in stop_list[i], 'HANKAR' in stop_list[i], 'TFCO OVErijse' in stop_list[i], 'TFCO VDK' in stop_list[i]],):
                cap.append(3)
            else:
                cap.append(2)

        V = [0] + N
#         q = dict(zip(V, cap))
        q = {i : 1 for i in N} # dictionary of demands

        # create set of arcs
        A = [(i,j) for i in V for j in V if i!=j]
     
        # solve using CPLEX
        mdl = Model('CVRP')
        
#         # gap tolerance
#         mdl.parameters.mip.tolerances.mipgap=0.02;
#         mdl.parameters.emphasis.mip=3;
#         mdl.parameters.mip.strategy.search=1;

        x = mdl.binary_var_dict(A, name='x')
        u = mdl.continuous_var_dict(N, ub=Q, name='u') 
        
# 1MM ------------------------------------------------

        if order == 'first':
            mdl.minimize(mdl.sum((proba_matrix[i,j])*x[i,j] for i,j in A))
    
# ----------------------------------------------------
        
# 2MM ------------------------------------------------
        
        elif order == 'second':
            # objective function
            depot_list = []
            square_list = []
            for i in V:
                if i == 0: # depot
                    for j in V:
                        if j != i:
    #                         print("adding depot cost:",i,j, proba_matrix[0,0,j])
                            depot_list.append( proba_matrix[0,0,j]*x[0,j] ) # np.log(proba_matrix[0,0,j])*x[0,j]
        
                            for k in V:
                                if k == i or k == j:
                                    continue
                                square_list.append( proba_matrix[0,j,k]*x[0,j]*x[j,k] )
                            
                else: # not depot
                    for j in V:
                        if j == 0 or j == i:
                            continue # depot and loop not allowed
                        for k in V:
                            if k == i or k == j:
                                continue # depot allowed, loops not
#                             if k == 0:
#                                 depot_list.append( proba_matrix[i,j,0]*x[i,j]*x[j,0] ) # np.log(proba_matrix[i,j,0])*x[j,0] 
    #                         print("adding sq cost:",i,j,k, proba_matrix[i,j,k])
                            square_list.append( proba_matrix[i,j,k]*x[i,j]*x[j,k] ) # np.log(proba_matrix[i,j,k])*x[i,j]*x[j,k] 
    #                         square_list.append( proba_matrix[i,j,k]*(x[i,j]*x[i,j]+x[j,k]*x[j,k]) )
            mdl.minimize(mdl.sum(depot_list + square_list)) # depot_list + square_list

# OLD ------------------------------------------------

#             mdl.minimize(mdl.sum(proba_matrix[i,j,k]*x[i,j]*x[j,k] for i,j in A for j,k in A if i!=j and j!=k and k!=i))

        # constraints
        mdl.add_constraints(mdl.sum(x[i,j] for j in V if j!=i)==1 for i in N)
        mdl.add_constraints(mdl.sum(x[i,j] for i in V if i!=j)==1 for j in N)
        mdl.add_indicator_constraints(mdl.indicator_constraint(x[i,j], u[i]+q[j]==u[j]) for i,j in A if i!=0 and j!=0)
        mdl.add_constraints(u[i]>=q[i] for i in N)

        # fix number of routes
        mdl.add_constraints(mdl.sum(x[0,j] for j in N)<=rt_count for j in N)
        mdl.add_constraints(mdl.sum(x[0,j] for j in N)>=rt_count for j in N)

        # set time limit
#         mdl.parameters.timelimit = 3600*12 # 5 hours (86400 - 1 day)
        mdl.parameters.timelimit = 20 # 5 hours (86400 - 1 day)
        
        start = time.time()

        # set (log_output=True) to display solution
        solution = mdl.solve()
#         solution = mdl.solve(log_output=True)
#         print("gap:",mdl.MIP.get_mip_relative_gap())
#         print("gap:",mdl.get_solve_details().mip_relative_gap)
        gap = mdl.get_solve_details().mip_relative_gap
        
        end = time.time()

        sol_time = end-start
        
        print("solution time:",sol_time)

        print('mdl.objective_value =', mdl.objective_value)
        
        active_arcs = [a for a in A if x[a].solution_value>0.9]
        start_arcs = [a for a in active_arcs if a[0]==0]

        res_plan = []
        for arc in start_arcs:    
            res_route = [0,arc[1]]
            while res_route[-1]!=0:
                for a in active_arcs:
                    if res_route[-1]==a[0]:
                        res_route.append(a[1])
            res_plan.append(res_route)

        # convert stops in actual to correspond to stops in test
        
        d = dict(zip(idx_list, V))
#         print(idx_list)
        
        new_actual = []
        
        for route in actual:
            new_route = []
            for s in route:
                new_route.append(d[s])
            new_actual.append(new_route)
        
        actual_for_comp = [route[1:-1] for route in new_actual]
        return [res_plan, str(solution.solve_status).split(".")[1].split("_")[0], actual_for_comp, gap, sol_time]

In [5]:
file_list = [file for file in sorted(glob.glob("*.csv")) if 'DIMANCHE' in file]
n_test = 7
weighing = 'unif'

In [6]:
# initialize df of results 
results_df = pd.DataFrame()
results_df = pd.DataFrame(columns = ['Date','Target','Solution', 'Status', 'SD','%SD','AD','%AD', 'Gap', 'Time'])

for x in range(len(file_list)-n_test, len(file_list)):
# for x in range(len(file_list)-1, len(file_list)):
    
    if weighing == 'fxd':
        train = file_list[:-n_test]
    else:
        train = file_list[:x]
    test = file_list[x]
    
    print(test)
    
    proba_matrix1, s_list1, idx_list1 = create_pmatrix(train, test, weighing)
        
    all_stops_list = s_list1

    actual = convert(get_route(test),all_stops_list)

    n = len(set(flatten(actual)))-1 # len(stop_list)-1
        
    N = [i for i in range(1, n+1)]
    rt_count = len(actual)   
    
    Q = n
    
    V = [0] + N

    q = {i : 1 for i in N} # dictionary of demands

    # create set of arcs
    A = [(i,j) for i in V for j in V if i!=j]
     
    # solve using CPLEX
    mdl = Model('CVRP')

    x = mdl.binary_var_dict(A, name='x')
    u = mdl.continuous_var_dict(N, ub=Q, name='u') 
            
    mdl.minimize(mdl.sum((proba_matrix1[i,j])*x[i,j] for i,j in A))
    
    # constraints
    mdl.add_constraints(mdl.sum(x[i,j] for j in V if j!=i)==1 for i in N)
    mdl.add_constraints(mdl.sum(x[i,j] for i in V if i!=j)==1 for j in N)
    mdl.add_indicator_constraints(mdl.indicator_constraint(x[i,j], u[i]+q[j]==u[j]) for i,j in A if i!=0 and j!=0)
    mdl.add_constraints(u[i]>=q[i] for i in N)

    # fix number of routes
    mdl.add_constraints(mdl.sum(x[0,j] for j in N)<=rt_count for j in N)
    mdl.add_constraints(mdl.sum(x[0,j] for j in N)>=rt_count for j in N)

    # set time limit
#         mdl.parameters.timelimit = 3600*12 # 5 hours (86400 - 1 day)
    mdl.parameters.timelimit = 20 # 5 hours (86400 - 1 day)
        
    start = time.time()

    # set (log_output=True) to display solution
    solution = mdl.solve()
#         solution = mdl.solve(log_output=True)
#         print("gap:",mdl.MIP.get_mip_relative_gap())
#         print("gap:",mdl.get_solve_details().mip_relative_gap)
    gap = mdl.get_solve_details().mip_relative_gap
        
    end = time.time()

    sol_time = end-start
        
    print("solution time:",sol_time)

    print('mdl.objective_value =', mdl.objective_value)
        
    active_arcs = [a for a in A if x[a].solution_value>0.9]
    start_arcs = [a for a in active_arcs if a[0]==0]

    res_plan = []
    for arc in start_arcs:    
        res_route = [0,arc[1]]
        while res_route[-1]!=0:
            for a in active_arcs:
                if res_route[-1]==a[0]:
                    res_route.append(a[1])
        res_plan.append(res_route)
        
    d = dict(zip(idx_list1, V))

        
    new_actual = []
        
    for route in actual:
        new_route = []
        for s in route:
            new_route.append(d[s])
        new_actual.append(new_route)
        
    actual_for_comp = [route[1:-1] for route in new_actual]
    
    results_df.loc[len(results_df)] = [test.split(" ")[0], actual_for_comp, res_plan, 
                                      str(solution.solve_status).split(".")[1].split("_")[0],
                                      eval_sd(actual_for_comp,res_plan)[1], eval_sd(actual_for_comp,res_plan)[2],
                                      eval_ad(actual_for_comp,res_plan)[1], eval_ad(actual_for_comp,res_plan)[2], gap, sol_time]

190210 DIMANCHE.csv
solution time: 0.1520707607269287
mdl.objective_value = 78.71248098889728
190217 DIMANCHE.csv
solution time: 0.1509871482849121
mdl.objective_value = 80.04984612261458
190224 DIMANCHE.csv
solution time: 0.10741519927978516
mdl.objective_value = 66.38134629667141
190303 DIMANCHE.csv
solution time: 0.18619251251220703
mdl.objective_value = 79.81095421329441
190310 DIMANCHE.csv
solution time: 0.16695928573608398
mdl.objective_value = 81.91325151439412
190317 DIMANCHE.csv
solution time: 0.17744874954223633
mdl.objective_value = 70.53070552635452
190324 DIMANCHE.csv
solution time: 0.18153667449951172
mdl.objective_value = 80.21778255419703


In [7]:
results_df

,Date,Target,Solution,Status,SD,%SD,AD,%AD,Gap,Time
0,190210,"[[0, 25, 1, 26, 2, 14, 3, 0], [0, 5, 18, 0], [...","[[0, 1, 22, 12, 9, 0], [0, 3, 0], [0, 5, 18, 0...",OPTIMAL,11,0.379310,25,0.714286,0.0,0.152071
1,190217,"[[0, 7, 1, 21, 27, 2, 15, 3, 0], [0, 5, 19, 0]...","[[0, 1, 24, 13, 10, 0], [0, 3, 0], [0, 5, 19, ...",OPTIMAL,17,0.566667,25,0.694444,0.0,0.150987
2,190224,"[[0, 1, 24, 2, 13, 3, 0], [0, 5, 17, 0], [0, 9...","[[0, 1, 21, 11, 8, 0], [0, 3, 0], [0, 5, 17, 0...",OPTIMAL,10,0.384615,20,0.625000,0.0,0.107415
3,190303,"[[0, 6, 1, 26, 27, 2, 15, 3, 0], [0, 5, 19, 0]...","[[0, 1, 23, 12, 9, 0], [0, 3, 0], [0, 5, 19, 0...",OPTIMAL,16,0.533333,25,0.694444,0.0,0.186193
4,190310,"[[0, 1, 27, 2, 15, 3, 0], [0, 5, 19, 0], [0, 1...","[[0, 1, 24, 13, 10, 0], [0, 3, 0], [0, 5, 19, ...",OPTIMAL,14,0.451613,24,0.648649,0.0,0.166959
5,190317,"[[0, 1, 25, 2, 14, 3, 0], [0, 5, 18, 0], [0, 1...","[[0, 1, 22, 12, 9, 0], [0, 5, 18, 0], [0, 8, 1...",OPTIMAL,10,0.357143,20,0.588235,0.0,0.177449
6,190324,"[[0, 6, 1, 27, 26, 2, 15, 3, 0], [0, 5, 19, 0]...","[[0, 1, 26, 25, 21, 29, 20, 0], [0, 5, 19, 0],...",OPTIMAL,12,0.387097,11,0.297297,0.0,0.181537


In [8]:
file_list = [file for file in sorted(glob.glob("*.csv")) if 'DIMANCHE' in file]
n_test = 7
weighing = 'wtd'

In [9]:
# initialize df of results 
results_df = pd.DataFrame()
results_df = pd.DataFrame(columns = ['Date','Target','Solution', 'Status', 'SD','%SD','AD','%AD', 'Gap', 'Time'])

for x in range(len(file_list)-n_test, len(file_list)):
# for x in range(len(file_list)-1, len(file_list)):
    
    if weighing == 'fxd':
        train = file_list[:-n_test]
    else:
        train = file_list[:x]
    test = file_list[x]
    
    print(test)
    
    proba_matrix1, s_list1, idx_list1 = create_pmatrix(train, test, weighing)
        
    all_stops_list = s_list1

    actual = convert(get_route(test),all_stops_list)

    n = len(set(flatten(actual)))-1 # len(stop_list)-1
        
    N = [i for i in range(1, n+1)]
    rt_count = len(actual)   
    
    Q = n
    
    V = [0] + N

    q = {i : 1 for i in N} # dictionary of demands

    # create set of arcs
    A = [(i,j) for i in V for j in V if i!=j]
     
    # solve using CPLEX
    mdl = Model('CVRP')

    x = mdl.binary_var_dict(A, name='x')
    u = mdl.continuous_var_dict(N, ub=Q, name='u') 
            
    mdl.minimize(mdl.sum((proba_matrix1[i,j])*x[i,j] for i,j in A))
    
    # constraints
    mdl.add_constraints(mdl.sum(x[i,j] for j in V if j!=i)==1 for i in N)
    mdl.add_constraints(mdl.sum(x[i,j] for i in V if i!=j)==1 for j in N)
    mdl.add_indicator_constraints(mdl.indicator_constraint(x[i,j], u[i]+q[j]==u[j]) for i,j in A if i!=0 and j!=0)
    mdl.add_constraints(u[i]>=q[i] for i in N)

    # fix number of routes
    mdl.add_constraints(mdl.sum(x[0,j] for j in N)<=rt_count for j in N)
    mdl.add_constraints(mdl.sum(x[0,j] for j in N)>=rt_count for j in N)

    # set time limit
#         mdl.parameters.timelimit = 3600*12 # 5 hours (86400 - 1 day)
    mdl.parameters.timelimit = 20 # 5 hours (86400 - 1 day)
        
    start = time.time()

    # set (log_output=True) to display solution
    solution = mdl.solve()
#         solution = mdl.solve(log_output=True)
#         print("gap:",mdl.MIP.get_mip_relative_gap())
#         print("gap:",mdl.get_solve_details().mip_relative_gap)
    gap = mdl.get_solve_details().mip_relative_gap
        
    end = time.time()

    sol_time = end-start
        
    print("solution time:",sol_time)

    print('mdl.objective_value =', mdl.objective_value)
        
    active_arcs = [a for a in A if x[a].solution_value>0.9]
    start_arcs = [a for a in active_arcs if a[0]==0]

    res_plan = []
    for arc in start_arcs:    
        res_route = [0,arc[1]]
        while res_route[-1]!=0:
            for a in active_arcs:
                if res_route[-1]==a[0]:
                    res_route.append(a[1])
        res_plan.append(res_route)
        
    d = dict(zip(idx_list1, V))

        
    new_actual = []
        
    for route in actual:
        new_route = []
        for s in route:
            new_route.append(d[s])
        new_actual.append(new_route)
        
    actual_for_comp = [route[1:-1] for route in new_actual]
    
    results_df.loc[len(results_df)] = [test.split(" ")[0], actual_for_comp, res_plan, 
                                      str(solution.solve_status).split(".")[1].split("_")[0],
                                      eval_sd(actual_for_comp,res_plan)[1], eval_sd(actual_for_comp,res_plan)[2],
                                      eval_ad(actual_for_comp,res_plan)[1], eval_ad(actual_for_comp,res_plan)[2], gap, sol_time]

190210 DIMANCHE.csv
solution time: 0.125807523727417
mdl.objective_value = 85.83151770893035
190217 DIMANCHE.csv
solution time: 0.19807839393615723
mdl.objective_value = 87.60213768675288
190224 DIMANCHE.csv
solution time: 0.09162354469299316
mdl.objective_value = 70.71690434793248
190303 DIMANCHE.csv
solution time: 0.20401525497436523
mdl.objective_value = 82.73357422127265
190310 DIMANCHE.csv
solution time: 0.15016865730285645
mdl.objective_value = 83.77594780011617
190317 DIMANCHE.csv
solution time: 0.11272811889648438
mdl.objective_value = 70.18493131900887
190324 DIMANCHE.csv
solution time: 0.15139269828796387
mdl.objective_value = 80.02169589894949


In [10]:
results_df

,Date,Target,Solution,Status,SD,%SD,AD,%AD,Gap,Time
0,190210,"[[0, 25, 1, 26, 2, 14, 3, 0], [0, 5, 18, 0], [...","[[0, 1, 0], [0, 5, 18, 0], [0, 10, 25, 28, 8, ...",OPTIMAL,11,0.379310,22,0.628571,0.0,0.125808
1,190217,"[[0, 7, 1, 21, 27, 2, 15, 3, 0], [0, 5, 19, 0]...","[[0, 1, 13, 24, 12, 25, 10, 0], [0, 5, 19, 0],...",OPTIMAL,9,0.300000,15,0.416667,0.0,0.198078
2,190224,"[[0, 1, 24, 2, 13, 3, 0], [0, 5, 17, 0], [0, 9...","[[0, 1, 24, 2, 13, 3, 0], [0, 5, 17, 0], [0, 7...",OPTIMAL,2,0.076923,4,0.125000,0.0,0.091624
3,190303,"[[0, 6, 1, 26, 27, 2, 15, 3, 0], [0, 5, 19, 0]...","[[0, 1, 26, 2, 15, 3, 0], [0, 5, 19, 0], [0, 8...",OPTIMAL,3,0.100000,8,0.222222,0.0,0.204015
4,190310,"[[0, 1, 27, 2, 15, 3, 0], [0, 5, 19, 0], [0, 1...","[[0, 1, 27, 2, 15, 3, 0], [0, 5, 19, 0], [0, 9...",OPTIMAL,3,0.096774,9,0.243243,0.0,0.150169
5,190317,"[[0, 1, 25, 2, 14, 3, 0], [0, 5, 18, 0], [0, 1...","[[0, 1, 25, 2, 14, 3, 0], [0, 5, 18, 0], [0, 8...",OPTIMAL,0,0.000000,0,0.000000,0.0,0.112728
6,190324,"[[0, 6, 1, 27, 26, 2, 15, 3, 0], [0, 5, 19, 0]...","[[0, 1, 26, 2, 15, 3, 0], [0, 5, 19, 0], [0, 8...",OPTIMAL,2,0.064516,4,0.108108,0.0,0.151393


In [11]:
file_list = [file for file in sorted(glob.glob("*.csv")) if 'DIMANCHE' in file]
n_test = 7
weighing = 'exp'

In [12]:
# initialize df of results 
results_df = pd.DataFrame()
results_df = pd.DataFrame(columns = ['Date','Target','Solution', 'Status', 'SD','%SD','AD','%AD', 'Gap', 'Time'])

for x in range(len(file_list)-n_test, len(file_list)):
# for x in range(len(file_list)-1, len(file_list)):
    
    if weighing == 'fxd':
        train = file_list[:-n_test]
    else:
        train = file_list[:x]
    test = file_list[x]
    
    print(test)
    
    proba_matrix1, s_list1, idx_list1 = create_pmatrix(train, test, weighing)
        
    all_stops_list = s_list1

    actual = convert(get_route(test),all_stops_list)

    n = len(set(flatten(actual)))-1 # len(stop_list)-1
        
    N = [i for i in range(1, n+1)]
    rt_count = len(actual)   
    
    Q = n
    
    V = [0] + N

    q = {i : 1 for i in N} # dictionary of demands

    # create set of arcs
    A = [(i,j) for i in V for j in V if i!=j]
     
    # solve using CPLEX
    mdl = Model('CVRP')

    x = mdl.binary_var_dict(A, name='x')
    u = mdl.continuous_var_dict(N, ub=Q, name='u') 
            
    mdl.minimize(mdl.sum((proba_matrix1[i,j])*x[i,j] for i,j in A))
    
    # constraints
    mdl.add_constraints(mdl.sum(x[i,j] for j in V if j!=i)==1 for i in N)
    mdl.add_constraints(mdl.sum(x[i,j] for i in V if i!=j)==1 for j in N)
    mdl.add_indicator_constraints(mdl.indicator_constraint(x[i,j], u[i]+q[j]==u[j]) for i,j in A if i!=0 and j!=0)
    mdl.add_constraints(u[i]>=q[i] for i in N)

    # fix number of routes
    mdl.add_constraints(mdl.sum(x[0,j] for j in N)<=rt_count for j in N)
    mdl.add_constraints(mdl.sum(x[0,j] for j in N)>=rt_count for j in N)

    # set time limit
#         mdl.parameters.timelimit = 3600*12 # 5 hours (86400 - 1 day)
    mdl.parameters.timelimit = 20 # 5 hours (86400 - 1 day)
        
    start = time.time()

    # set (log_output=True) to display solution
    solution = mdl.solve()
#         solution = mdl.solve(log_output=True)
#         print("gap:",mdl.MIP.get_mip_relative_gap())
#         print("gap:",mdl.get_solve_details().mip_relative_gap)
    gap = mdl.get_solve_details().mip_relative_gap
        
    end = time.time()

    sol_time = end-start
        
    print("solution time:",sol_time)

    print('mdl.objective_value =', mdl.objective_value)
        
    active_arcs = [a for a in A if x[a].solution_value>0.9]
    start_arcs = [a for a in active_arcs if a[0]==0]

    res_plan = []
    for arc in start_arcs:    
        res_route = [0,arc[1]]
        while res_route[-1]!=0:
            for a in active_arcs:
                if res_route[-1]==a[0]:
                    res_route.append(a[1])
        res_plan.append(res_route)
        
    d = dict(zip(idx_list1, V))

        
    new_actual = []
        
    for route in actual:
        new_route = []
        for s in route:
            new_route.append(d[s])
        new_actual.append(new_route)
        
    actual_for_comp = [route[1:-1] for route in new_actual]
    
    results_df.loc[len(results_df)] = [test.split(" ")[0], actual_for_comp, res_plan, 
                                      str(solution.solve_status).split(".")[1].split("_")[0],
                                      eval_sd(actual_for_comp,res_plan)[1], eval_sd(actual_for_comp,res_plan)[2],
                                      eval_ad(actual_for_comp,res_plan)[1], eval_ad(actual_for_comp,res_plan)[2], gap, sol_time]

190210 DIMANCHE.csv
solution time: 0.11690998077392578
mdl.objective_value = 101.29423496407944
190217 DIMANCHE.csv
solution time: 0.13352036476135254
mdl.objective_value = 104.75995229084626
190224 DIMANCHE.csv
solution time: 0.09225320816040039
mdl.objective_value = 89.15758697889143
190303 DIMANCHE.csv
solution time: 0.1175684928894043
mdl.objective_value = 105.85949091183281
190310 DIMANCHE.csv
solution time: 0.15574169158935547
mdl.objective_value = 109.73170260064532
190317 DIMANCHE.csv
solution time: 0.11285996437072754
mdl.objective_value = 96.67122828313154
190324 DIMANCHE.csv
solution time: 0.13738131523132324
mdl.objective_value = 109.26204320828992


In [13]:
results_df

,Date,Target,Solution,Status,SD,%SD,AD,%AD,Gap,Time
0,190210,"[[0, 25, 1, 26, 2, 14, 3, 0], [0, 5, 18, 0], [...","[[0, 1, 26, 2, 14, 3, 0], [0, 5, 18, 0], [0, 8...",OPTIMAL,4,0.137931,6,0.171429,0.000000,0.116910
1,190217,"[[0, 7, 1, 21, 27, 2, 15, 3, 0], [0, 5, 19, 0]...","[[0, 1, 27, 2, 15, 3, 0], [0, 5, 19, 0], [0, 9...",OPTIMAL,4,0.133333,6,0.166667,0.000000,0.133520
2,190224,"[[0, 1, 24, 2, 13, 3, 0], [0, 5, 17, 0], [0, 9...","[[0, 1, 24, 2, 13, 3, 0], [0, 5, 17, 0], [0, 7...",OPTIMAL,1,0.038462,3,0.093750,0.000000,0.092253
3,190303,"[[0, 6, 1, 26, 27, 2, 15, 3, 0], [0, 5, 19, 0]...","[[0, 1, 26, 2, 15, 3, 0], [0, 5, 19, 0], [0, 8...",OPTIMAL,3,0.100000,7,0.194444,0.000000,0.117568
4,190310,"[[0, 1, 27, 2, 15, 3, 0], [0, 5, 19, 0], [0, 1...","[[0, 5, 19, 0], [0, 8, 1, 27, 2, 15, 3, 0], [0...",OPTIMAL,3,0.096774,9,0.243243,0.000000,0.155742
5,190317,"[[0, 1, 25, 2, 14, 3, 0], [0, 5, 18, 0], [0, 1...","[[0, 1, 25, 2, 14, 3, 0], [0, 5, 18, 0], [0, 8...",OPTIMAL,0,0.000000,0,0.000000,0.000000,0.112860
6,190324,"[[0, 6, 1, 27, 26, 2, 15, 3, 0], [0, 5, 19, 0]...","[[0, 1, 26, 2, 15, 3, 0], [0, 5, 19, 0], [0, 8...",OPTIMAL,2,0.064516,4,0.108108,0.000005,0.137381


## Second order

In [ ]:
# mdl.maximize(mdl.sum(c[i,j,k]*x[i,j]*x[j,k] for i,j in A for j,k in A if i!=j))

In [16]:
# initialize df of results 
results_df = pd.DataFrame()
results_df = pd.DataFrame(columns = ['Date','Target','Solution', 'Status', 'SD','%SD','AD','%AD', 'Gap', 'Time'])

for x in range(len(file_list)-n_test, len(file_list)):
# for x in range(len(file_list)-1, len(file_list)):
    
    if weighing == 'fxd':
        train = file_list[:-n_test]
    else:
        train = file_list[:x]
    test = file_list[x]
    
    print(test)
    
    proba_matrix1, s_list1, idx_list1 = create3d_mat(train, test, weighing)
        
    all_stops_list = s_list1

    actual = convert(get_route(test),all_stops_list)

    n = len(set(flatten(actual)))-1 # len(stop_list)-1
        
    N = [i for i in range(1, n+1)]
    rt_count = len(actual)   
    
    Q = n
    
    V = [0] + N

    q = {i : 1 for i in N} # dictionary of demands

    # create set of arcs
    A = [(i,j) for i in V for j in V if i!=j]
     
    # solve using CPLEX
    mdl = Model('CVRP')

    x = mdl.binary_var_dict(A, name='x')
    u = mdl.continuous_var_dict(N, ub=Q, name='u') 
            
#     mdl.minimize(mdl.sum((proba_matrix1[i,j])*x[i,j] for i,j in A))
    mdl.minimize(mdl.sum(proba_matrix1[i,j,k]*x[i,j]*x[j,k] for i,j in A for j,k in A if i!=j))
    
    # constraints
    mdl.add_constraints(mdl.sum(x[i,j] for j in V if j!=i)==1 for i in N)
    mdl.add_constraints(mdl.sum(x[i,j] for i in V if i!=j)==1 for j in N)
    mdl.add_indicator_constraints(mdl.indicator_constraint(x[i,j], u[i]+q[j]==u[j]) for i,j in A if i!=0 and j!=0)
    mdl.add_constraints(u[i]>=q[i] for i in N)

    # fix number of routes
    mdl.add_constraints(mdl.sum(x[0,j] for j in N)<=rt_count for j in N)
    mdl.add_constraints(mdl.sum(x[0,j] for j in N)>=rt_count for j in N)

    # set time limit
#         mdl.parameters.timelimit = 3600*12 # 5 hours (86400 - 1 day)
    mdl.parameters.timelimit = 20 # 5 hours (86400 - 1 day)
        
    start = time.time()

    # set (log_output=True) to display solution
    solution = mdl.solve()
#         solution = mdl.solve(log_output=True)
#         print("gap:",mdl.MIP.get_mip_relative_gap())
#         print("gap:",mdl.get_solve_details().mip_relative_gap)
    gap = mdl.get_solve_details().mip_relative_gap
        
    end = time.time()

    sol_time = end-start
        
    print("solution time:",sol_time)

    print('mdl.objective_value =', mdl.objective_value)
        
    active_arcs = [a for a in A if x[a].solution_value>0.9]
    start_arcs = [a for a in active_arcs if a[0]==0]

    res_plan = []
    for arc in start_arcs:    
        res_route = [0,arc[1]]
        while res_route[-1]!=0:
            for a in active_arcs:
                if res_route[-1]==a[0]:
                    res_route.append(a[1])
        res_plan.append(res_route)
        
    d = dict(zip(idx_list1, V))

        
    new_actual = []
        
    for route in actual:
        new_route = []
        for s in route:
            new_route.append(d[s])
        new_actual.append(new_route)
        
    actual_for_comp = [route[1:-1] for route in new_actual]
    
    results_df.loc[len(results_df)] = [test.split(" ")[0], actual_for_comp, res_plan, 
                                      str(solution.solve_status).split(".")[1].split("_")[0],
                                      eval_sd(actual_for_comp,res_plan)[1], eval_sd(actual_for_comp,res_plan)[2],
                                      eval_ad(actual_for_comp,res_plan)[1], eval_ad(actual_for_comp,res_plan)[2], gap, sol_time]

190210 DIMANCHE.csv
solution time: 20.124558210372925
mdl.objective_value = 19030.497812441594
190217 DIMANCHE.csv
solution time: 20.16767907142639
mdl.objective_value = 11641.575918016277
190224 DIMANCHE.csv
solution time: 20.12532877922058
mdl.objective_value = 7564.6443439106
190303 DIMANCHE.csv
solution time: 20.11796736717224
mdl.objective_value = 7811.841701510596
190310 DIMANCHE.csv
solution time: 20.115654945373535
mdl.objective_value = 15909.07684261263
190317 DIMANCHE.csv
solution time: 20.118430614471436
mdl.objective_value = 8101.239617474899
190324 DIMANCHE.csv
solution time: 20.130167245864868
mdl.objective_value = 8960.683595116707


In [17]:
results_df

,Date,Target,Solution,Status,SD,%SD,AD,%AD,Gap,Time
0,190210,"[[0, 25, 1, 26, 2, 14, 3, 0], [0, 5, 18, 0], [...","[[0, 3, 0], [0, 8, 7, 0], [0, 12, 6, 0], [0, 1...",FEASIBLE,17,0.586207,29,0.828571,1.0,20.124558
1,190217,"[[0, 7, 1, 21, 27, 2, 15, 3, 0], [0, 5, 19, 0]...","[[0, 7, 22, 4, 11, 0], [0, 10, 3, 2, 29, 23, 0...",FEASIBLE,18,0.600000,33,0.916667,1.0,20.167679
2,190224,"[[0, 1, 24, 2, 13, 3, 0], [0, 5, 17, 0], [0, 9...","[[0, 1, 24, 11, 16, 0], [0, 2, 0], [0, 5, 17, ...",FEASIBLE,14,0.538462,24,0.750000,1.0,20.125329
3,190303,"[[0, 6, 1, 26, 27, 2, 15, 3, 0], [0, 5, 19, 0]...","[[0, 3, 21, 12, 24, 5, 13, 0], [0, 11, 23, 14,...",FEASIBLE,23,0.766667,36,1.000000,1.0,20.117967
4,190310,"[[0, 1, 27, 2, 15, 3, 0], [0, 5, 19, 0], [0, 1...","[[0, 4, 23, 29, 22, 27, 26, 7, 20, 11, 0], [0,...",FEASIBLE,17,0.548387,34,0.918919,1.0,20.115655
5,190317,"[[0, 1, 25, 2, 14, 3, 0], [0, 5, 18, 0], [0, 1...","[[0, 1, 20, 27, 0], [0, 3, 11, 19, 22, 0], [0,...",FEASIBLE,16,0.571429,31,0.911765,1.0,20.118431
6,190324,"[[0, 6, 1, 27, 26, 2, 15, 3, 0], [0, 5, 19, 0]...","[[0, 9, 29, 30, 5, 0], [0, 11, 0], [0, 12, 13,...",FEASIBLE,21,0.677419,35,0.945946,1.0,20.130167


In [18]:
# initialize df of results 
results_df = pd.DataFrame()
results_df = pd.DataFrame(columns = ['Date','Target','Solution', 'Status', 'SD','%SD','AD','%AD', 'Gap', 'Time'])

for x in range(len(file_list)-n_test, len(file_list)):
# for x in range(len(file_list)-1, len(file_list)):
    
    if weighing == 'fxd':
        train = file_list[:-n_test]
    else:
        train = file_list[:x]
    test = file_list[x]
    
    print(test)
    
    proba_matrix1, s_list1, idx_list1 = create3d_mat(train, test, weighing)
        
    all_stops_list = s_list1

    actual = convert(get_route(test),all_stops_list)

    n = len(set(flatten(actual)))-1 # len(stop_list)-1
        
    N = [i for i in range(1, n+1)]
    rt_count = len(actual)   
    
    Q = n
    
    V = [0] + N

    q = {i : 1 for i in N} # dictionary of demands

    # create set of arcs
    A = [(i,j) for i in V for j in V if i!=j]
     
    # solve using CPLEX
    mdl = Model('CVRP')

    x = mdl.binary_var_dict(A, name='x')
    u = mdl.continuous_var_dict(N, ub=Q, name='u') 
            
#     mdl.minimize(mdl.sum((proba_matrix1[i,j])*x[i,j] for i,j in A))
    mdl.minimize(mdl.sum(proba_matrix1[i,j,k]*x[i,j]*x[j,k] for i,j in A for j,k in A if i!=j))
    
    # constraints
    mdl.add_constraints(mdl.sum(x[i,j] for j in V if j!=i)==1 for i in N)
    mdl.add_constraints(mdl.sum(x[i,j] for i in V if i!=j)==1 for j in N)
    mdl.add_indicator_constraints(mdl.indicator_constraint(x[i,j], u[i]+q[j]==u[j]) for i,j in A if i!=0 and j!=0)
    mdl.add_constraints(u[i]>=q[i] for i in N)

    # fix number of routes
    mdl.add_constraints(mdl.sum(x[0,j] for j in N)<=rt_count for j in N)
    mdl.add_constraints(mdl.sum(x[0,j] for j in N)>=rt_count for j in N)

    # set time limit
#         mdl.parameters.timelimit = 3600*12 # 5 hours (86400 - 1 day)
    mdl.parameters.timelimit = 60 # 5 hours (86400 - 1 day)
        
    start = time.time()

    # set (log_output=True) to display solution
    solution = mdl.solve()
#         solution = mdl.solve(log_output=True)
#         print("gap:",mdl.MIP.get_mip_relative_gap())
#         print("gap:",mdl.get_solve_details().mip_relative_gap)
    gap = mdl.get_solve_details().mip_relative_gap
        
    end = time.time()

    sol_time = end-start
        
    print("solution time:",sol_time)

    print('mdl.objective_value =', mdl.objective_value)
        
    active_arcs = [a for a in A if x[a].solution_value>0.9]
    start_arcs = [a for a in active_arcs if a[0]==0]

    res_plan = []
    for arc in start_arcs:    
        res_route = [0,arc[1]]
        while res_route[-1]!=0:
            for a in active_arcs:
                if res_route[-1]==a[0]:
                    res_route.append(a[1])
        res_plan.append(res_route)
        
    d = dict(zip(idx_list1, V))

        
    new_actual = []
        
    for route in actual:
        new_route = []
        for s in route:
            new_route.append(d[s])
        new_actual.append(new_route)
        
    actual_for_comp = [route[1:-1] for route in new_actual]
    
    results_df.loc[len(results_df)] = [test.split(" ")[0], actual_for_comp, res_plan, 
                                      str(solution.solve_status).split(".")[1].split("_")[0],
                                      eval_sd(actual_for_comp,res_plan)[1], eval_sd(actual_for_comp,res_plan)[2],
                                      eval_ad(actual_for_comp,res_plan)[1], eval_ad(actual_for_comp,res_plan)[2], gap, sol_time]

190210 DIMANCHE.csv
solution time: 60.148388385772705
mdl.objective_value = 7235.5363791256395
190217 DIMANCHE.csv
solution time: 60.191287994384766
mdl.objective_value = 8442.915926259642
190224 DIMANCHE.csv
solution time: 60.10194706916809
mdl.objective_value = 6068.3843875475195
190303 DIMANCHE.csv
solution time: 60.13038659095764
mdl.objective_value = 7786.08729535851
190310 DIMANCHE.csv
solution time: 60.14132213592529
mdl.objective_value = 8590.940954907102
190317 DIMANCHE.csv
solution time: 60.07478904724121
mdl.objective_value = 7034.018958989436
190324 DIMANCHE.csv
solution time: 60.15251135826111
mdl.objective_value = 8496.662950578631


In [19]:
results_df

,Date,Target,Solution,Status,SD,%SD,AD,%AD,Gap,Time
0,190210,"[[0, 25, 1, 26, 2, 14, 3, 0], [0, 5, 18, 0], [...","[[0, 3, 14, 0], [0, 7, 9, 11, 5, 6, 1, 8, 4, 2...",FEASIBLE,16,0.551724,29,0.828571,1.000000,60.148388
1,190217,"[[0, 7, 1, 21, 27, 2, 15, 3, 0], [0, 5, 19, 0]...","[[0, 2, 19, 26, 15, 5, 17, 4, 28, 9, 0], [0, 7...",FEASIBLE,20,0.666667,34,0.944444,1.000000,60.191288
2,190224,"[[0, 1, 24, 2, 13, 3, 0], [0, 5, 17, 0], [0, 9...","[[0, 2, 17, 14, 7, 11, 10, 0], [0, 3, 5, 0], [...",FEASIBLE,16,0.615385,29,0.906250,0.906037,60.101947
3,190303,"[[0, 6, 1, 26, 27, 2, 15, 3, 0], [0, 5, 19, 0]...","[[0, 2, 20, 16, 14, 0], [0, 6, 1, 9, 4, 12, 19...",FEASIBLE,22,0.733333,34,0.944444,1.000000,60.130387
4,190310,"[[0, 1, 27, 2, 15, 3, 0], [0, 5, 19, 0], [0, 1...","[[0, 6, 1, 18, 28, 14, 13, 0], [0, 11, 8, 0], ...",FEASIBLE,16,0.516129,31,0.837838,1.000000,60.141322
5,190317,"[[0, 1, 25, 2, 14, 3, 0], [0, 5, 18, 0], [0, 1...","[[0, 2, 10, 20, 3, 17, 7, 13, 0], [0, 6, 11, 1...",FEASIBLE,21,0.750000,34,1.000000,1.000000,60.074789
6,190324,"[[0, 6, 1, 27, 26, 2, 15, 3, 0], [0, 5, 19, 0]...","[[0, 2, 0], [0, 9, 26, 0], [0, 13, 19, 10, 14,...",FEASIBLE,23,0.741935,37,1.000000,1.000000,60.152511
